# Introduction

This notebook is a shortened version of a notebook that I wanted to use in my bachelor thesis.
Unfortunately I don't need the machine learning model anymore but I still wanted to share this approach...

I achieved an accuracy of around 70%

# Imports

In [10]:
import scipy
import numpy as np
import pandas as pd
import sklearn


from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB

from sklearn.preprocessing import LabelEncoder


# Load data

In [11]:
News_Titles_df = pd.read_csv("/Users/jan/Documents/Python_Projects/Mashine_Learning_Categories/News_Titles_Examples.csv", index_col=None, header=0)

News_Titles_df = News_Titles_df.drop_duplicates(subset="Title")

# EDA

In [17]:
News_Titles_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16147 entries, 0 to 16146
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Title      16147 non-null  object
 1   URL        16147 non-null  object
 2   Date_Info  16147 non-null  object
 3   News_page  16147 non-null  object
 4   Category   16147 non-null  object
dtypes: object(5)
memory usage: 756.9+ KB


In [12]:
News_Titles_df["Category"].value_counts()

Ausland         3608
Panorama        3116
Politik         2377
Wirtschaft      2112
Sport           1975
Wissenschaft    1365
Kultur          1074
Netzwelt         520
Name: Category, dtype: int64

# Encoding titles

In [13]:
titel_set = News_Titles_df[["Title","Category"]]
title_set_copy = titel_set.copy()
encoded_titel_set = Encoder(title_set_copy)

# Testing for best machine learning model

In [14]:
# Split-out validation dataset
dataset = encoded_titel_set
array = dataset.values
X = array[:,0]
y = array[:,1]
X_train, X_validation, Y_train, Y_validation = train_test_split(X, y, test_size=0.2, random_state=42)


X_train=X_train.reshape(-1, 1)
X_validation=X_validation.reshape(-1, 1)


# Spot Check Algorithms
models = []
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))


# evaluate each model in turn
results = []
names = []
for name, model in models:
    kfold = StratifiedKFold(n_splits=10, random_state=1, shuffle=True)
    cv_results = cross_val_score(model, X_train, Y_train, cv=kfold, scoring='accuracy')
    results.append(cv_results)
    names.append(name)

    print('%s: %f (%f)' % (name, cv_results.mean(), cv_results.std()))


LDA: 0.221259 (0.001688)
KNN: 0.432453 (0.006735)
CART: 0.413099 (0.013943)
NB: 0.219866 (0.002837)


# Testing Multinomial Naive Bayes algorithm 

In [15]:
data = titel_set

x = np.array(data["Title"])
y = np.array(data["Category"])

cv = CountVectorizer()
X = cv.fit_transform(x)
X_train, X_validation, y_train, Y_validation = train_test_split(X, y, test_size=0.2, random_state=42)

model = MultinomialNB()
model.fit(X_train,y_train)

predictions = model.predict(X_validation)
# Evaluate predictions
print(accuracy_score(Y_validation, predictions))
print(confusion_matrix(Y_validation, predictions))
print(classification_report(Y_validation, predictions))

0.7027863777089783
[[649   1   0  43  22   7   8   8]
 [ 52  87   0  32  23  17  10   3]
 [ 23   2  16  18  18   3  18   2]
 [ 87   4   2 452  45  10  17   5]
 [ 84   1   0  34 324   4  13   8]
 [ 21   2   0  21   4 355   3   1]
 [ 82   4   0  23  27   7 256   9]
 [ 54   2   0  34  25   4  13 131]]
              precision    recall  f1-score   support

     Ausland       0.62      0.88      0.73       738
      Kultur       0.84      0.39      0.53       224
    Netzwelt       0.89      0.16      0.27       100
    Panorama       0.69      0.73      0.71       622
     Politik       0.66      0.69      0.68       468
       Sport       0.87      0.87      0.87       407
  Wirtschaft       0.76      0.63      0.69       408
Wissenschaft       0.78      0.50      0.61       263

    accuracy                           0.70      3230
   macro avg       0.76      0.61      0.64      3230
weighted avg       0.73      0.70      0.69      3230



# Helper

In [16]:
def Encoder(df):
    columnsToEncode = list(df.select_dtypes(include=['category','object']))
    le = LabelEncoder()
    for feature in columnsToEncode:
        try:
            df[feature] = le.fit_transform(df[feature])
        except:
            print('Error encoding '+feature)
    return df